In [13]:
from Chemometrics.import_common import *
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler, Normalizer
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict

plt.rcParams['font.sans-serif'] = 'SimHei'
%matplotlib inline

In [2]:
Xy = pload('./洋河2018换柱子后Xy.p')
X_train, X_test, y_train, y_test = Xy['X_train'], Xy['X_test'], Xy['y_train'], Xy['y_test']

## 方案
1. 根据m/z差距（小于一定阈值归为一组）进行分组，并合并(m/z: mean, intensity: sum)
2. 建模。
    -  base: LR
    -  KNN
    -  SVM

## KNN 方案
1. loger-normalizer-standardscaler-PCA-KNN  
  loger, normalizer, standardscaler, PCA 可选

In [45]:
def pipe_report(pipe, param_grid, X_train, y_train, X_test, y_test, verbose=True):
    grid = GridSearchCV(pipe, param_grid=param_grid)
    with timer("train"):
        grid.fit(X_train, y_train)
    #交叉验证效果
    ypred_cv = cross_val_predict(grid, X_train, y_train)
    if verbose:
        print(classification_report(y_train, ypred_cv))
    return grid

In [46]:
def _loger(x):
    return np.log(1+x)
loger = FunctionTransformer(_loger)
pipe = make_pipeline(loger, 
                      Normalizer(), 
                      StandardScaler(), 
                      PCA(n_components=.99), 
                      KNeighborsClassifier())
param_grid = dict(kneighborsclassifier__n_neighbors=np.arange(2,11),
                  pca=[None, PCA(.99)],
                  functiontransformer=[None,loger],
                  normalizer=[None, Normalizer()],
                  standardscaler=[None,StandardScaler()])

In [48]:
KNN = pipe_report(pipe, param_grid, X_train, y_train, verbose=True)

train done in 2.99s
             precision    recall  f1-score   support

         优绵       0.56      0.69      0.62        13
         优降       0.60      0.46      0.52        13

avg / total       0.58      0.58      0.57        26



### SVM方案
Normalizer - StandardScaler - PCA(n_components=.99)- SVM  
SVM前面步骤可选

In [74]:
from sklearn.svm import SVC
svc_pipe = make_pipeline(Normalizer(), 
                      StandardScaler(), 
                      PCA(n_components=.99), 
                      SVC())
param_grid = dict(pca=[None, PCA(.99)],
                  normalizer=[None, Normalizer()],
                  standardscaler=[None, StandardScaler()],
                  svc__C=np.power(2.0, np.arange(-10,10)))
SVM = pipe_report(svc_pipe, param_grid, X_train, y_train, verbose=True)

train done in 2.86s
             precision    recall  f1-score   support

         优绵       0.70      0.54      0.61        13
         优降       0.62      0.77      0.69        13

avg / total       0.66      0.65      0.65        26

